# Ch9 & 10 실습: 가설검정의 함정과 인과관계 추론

지난 시간, 우리는 `statsmodels`를 이용해 다양한 회귀 모델을 만드는 법을 배웠습니다. 오늘은 한 걸음 더 나아가, 통계 분석 결과를 해석할 때 빠지기 쉬운 함정들을 직접 코드로 체험하고, '상관관계'를 넘어 '인과관계'에 다가가는 고급 분석 기법들을 실습해 봅니다.

**학습 목표:**

1.  **허위 상관(Spurious Correlation)**과 **중첩요인(Confounder)**의 개념을 이해하고, 다중회귀로 중첩요인의 효과를 통제하는 법을 배웁니다.
2.  **p-해킹**과 **표본 크기의 함정**이 왜 위험한지 시뮬레이션을 통해 직접 증명합니다.
3.  실험이 불가능한 상황에서 인과 효과를 추정하는 강력한 도구, **경향 점수 짝짓기(PSM)**와 **이중차분법(DiD)**을 직접 구현해 봅니다.


In [5]:
# 라이브러리와 폰트를 로드
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as stats
from sklearn.neighbors import NearestNeighbors

# MacOS에서는 기본 한글 폰트로 AppleGothic 사용
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False  # 마이너스 깨짐 방지

# 경고 메시지 무시
import warnings
warnings.filterwarnings('ignore')

## 문제 1 (난이도: 하): 허위 상관과 중첩요인 통제하기

> **🎯 목표:** '아이스크림 판매량과 익사자 수'의 관계처럼, **허위 상관(Spurious Correlation)**이 발생하는 원인인 **중첩요인(Confounder)**을 이해하고, 다중회귀분석을 통해 그 효과를 통제하는 방법을 실습합니다.

### 💡 핵심 개념:

**허위 상관**은 두 변수 사이에 직접적인 인과관계가 없지만, '기온'과 같은 제3의 변수(중첩요인)가 두 변수 모두에 영향을 주어 마치 관계가 있는 것처럼 보이는 현상입니다. **다중회귀분석**에 중첩요인을 함께 투입하면, 다른 변수들의 영향을 고정한 상태에서의 '순수한' 관계를 파악할 수 있습니다.

### 📌 수행 과제:

1.  가상의 데이터를 생성합니다: '기온'이 높아질수록 '아이스크림 판매량'과 '해수욕객 수'가 모두 증가하는 상황을 시뮬레이션합니다.
2.  **단순회귀분석**: '아이스크림 판매량'만으로 '해수욕객 수'를 예측하는 모델을 만듭니다. 계수가 유의미하게 나오는지 확인합니다.
3.  **다중회귀분석**: '아이스크림 판매량'과 '기온'을 모두 사용하여 '해수욕객 수'를 예측하는 모델을 만듭니다.
4.  두 모델의 결과를 비교하고, '아이스크림 판매량'의 계수(coef)와 p-value가 어떻게 변했는지 해석해 보세요.

In [ ]:
# 1. 가상 데이터 생성 (이 코드는 수정하지 마세요)
np.random.seed(42)
n_samples = 100
temperature = np.random.uniform(15, 35, n_samples)
ice_cream_sales = 2 * temperature + np.random.normal(0, 5, n_samples)
beach_visitors = 10 * temperature + np.random.normal(0, 20, n_samples)
df1 = pd.DataFrame({'기온': temperature, '아이스크림판매량': ice_cream_sales, '해수욕객수': beach_visitors})
print(df1.corr())

# 2. 단순회귀분석: 아이스크림판매량 -> 해수욕객수
print("\n--- 모델 1: 단순회귀분석 (아이스크림판매량 -> 해수욕객수) ---")
y = df1['해수욕객수']
# 독립변수로 '아이스크림판매량'만 선택하세요.
X_simple = df1[['아이스크림판매량']]
# X_simple에 상수항을 추가하세요.
X_simple_const = sm.add_constant(X_simple)
# OLS 모델을 y와 X_simple_const로 학습시키세요.
model_simple = sm.OLS(y, X_simple_const).fit()
print(model_simple.summary())

# 다중회귀분석: (아이스크림판매량, 기온) -> 해수욕객수
print("\n--- 모델 2: 다중회귀분석 (중첩요인 '기온' 통제) ---")
# 독립변수로 '아이스크림판매량'과 '기온'을 모두 선택하세요.
X_multi = df1[['아이스크림판매량', '기온']]
# X_multi에 상수항을 추가하세요.
X_multi_const = sm.add_constant(X_multi)
# OLS 모델을 y와 X_multi_const로 학습시키세요.
model_multi = sm.OLS(y, X_multi_const).fit()
print(model_multi.summary())

# 4. 결과 해석 (아래 주석에 직접 작성해 보세요)
# 모델 1에서 아이스크림판매량의 계수와 p-value: 
# coef = 4.5550, p < 0.001 → 아이스크림 판매량이 증가할수록 해수욕객수가 늘어난다는 '유의미한 관계'처럼 보임.
# R² = 0.753 → 해수욕객수 변동의 약 75%가 아이스크림 판매량으로 설명됨.
# 
# 모델 2에서 아이스크림판매량의 계수와 p-value: 
# 아이스크림 판매량: coef = -0.1087, p = 0.826 → 유의하지 않음
# 기온: coef = 10.2149, p < 0.001 → 매우 유의미, 기온이 올라갈수록 해수욕객수가 크게 증가
# R² = 0.880 → 설명력이 훨씬 좋아짐.
# 
# 결과가 변한 이유:
# 단순회귀에서는 '아이스크림 판매량이 해수욕객 수를 늘린다'는 잘못된 결론을 내릴 수 있음.
# 하지만 다중회귀로 기온을 통제하자, 아이스크림 효과는 사라지고 실제로는 '기온'이 두 변수 모두에 영향을 준 중첩요인(Confounder)임이 드러남.

                기온  아이스크림판매량     해수욕객수
기온        1.000000  0.927709  0.938317
아이스크림판매량  0.927709  1.000000  0.867599
해수욕객수     0.938317  0.867599  1.000000

--- 모델 1: 단순회귀분석 (아이스크림판매량 -> 해수욕객수) ---
                            OLS Regression Results                            
Dep. Variable:                  해수욕객수   R-squared:                       0.753
Model:                            OLS   Adj. R-squared:                  0.750
Method:                 Least Squares   F-statistic:                     298.3
Date:                Mon, 15 Sep 2025   Prob (F-statistic):           1.70e-31
Time:                        17:16:50   Log-Likelihood:                -486.59
No. Observations:                 100   AIC:                             977.2
Df Residuals:                      98   BIC:                             982.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
            

### 🤔 생각해 볼 문제

만약 '기온' 데이터를 수집하지 못했다면,  
단순회귀 결과만 보고 **“아이스크림 판매량이 많아질수록 해수욕객 수가 늘어난다 → 아이스크림이 해수욕객 수에 직접적인 영향을 준다”** 라는 잘못된 인과 결론에 도달했을 위험이 있다.  
사실은 기온이라는 제3의 요인(중첩요인, Confounder)이 두 변수 모두에 영향을 준 것이었는데, 이를 고려하지 않으면 허위 상관(spurious correlation)을 인과관계로 오해하게 된다.  

이 사례는 데이터 분석에서 **도메인 지식**이 왜 중요한지를 보여준다.  
- 단순히 통계적 유의성(p-value)이나 상관계수만 보고 판단하면 잘못된 해석에 빠질 수 있다.  
- 실제 맥락(여름에 기온이 올라가면 아이스크림도 더 팔리고 해수욕객도 늘어난다)을 이해하고 있어야, 기온 같은 변수를 수집하고 모델에 반영할 수 있다.  
- 따라서 **통계기법 + 데이터**만으로는 부족하며, **현상에 대한 배경지식(도메인 지식)**이 함께 있어야 올바른 분석과 해석이 가능하다.


## 문제 2 (난이도: 하): 표본 크기의 함정과 효과 크기의 중요성

> **🎯 목표:** 표본 크기(n)가 커지면 아주 미미한 차이도 통계적으로 유의(p<0.05)하게 되는 현상을 확인하고, 이 때문에 **효과 크기(Effect Size)**를 함께 확인해야 하는 이유를 이해합니다.

### 💡 핵심 개념:

가설검정에서 p-value는 '귀무가설이 맞다고 가정할 때, 현재 데이터와 같거나 더 극단적인 결과가 나올 확률'입니다. 표본이 커질수록 아주 작은 차이도 우연으로 보기 어려워지므로 p-value는 작아집니다. **효과 크기**는 이 차이가 '실질적으로 얼마나 의미 있는 크기인가'를 나타내는 지표로, p-value의 한계를 보완해 줍니다.

### 📌 수행 과제:

1.  평균이 아주 약간 다른(예: 170cm vs 170.1cm) 두 집단을 가정합니다.
2.  **작은 표본(n=30)**을 각 집단에서 추출하여 t-검정을 수행하고 p-value를 확인합니다.
3.  **큰 표본(n=10,000)**을 각 집단에서 추출하여 t-검정을 수행하고 p-value를 확인합니다.
4.  두 경우 모두에 대해 **효과 크기(Cohen's d)**를 직접 계산하고, p-value와 효과 크기가 어떻게 다른 메시지를 주는지 비교 분석하세요.


In [ ]:
# 효과 크기(Cohen's d) 계산 함수 (이 코드는 수정하지 마세요)
def cohen_d(x, y):
    nx, ny = len(x), len(y)
    dof = nx + ny - 2
    return (np.mean(x) - np.mean(y)) / np.sqrt(((nx-1)*np.std(x, ddof=1)**2 + (ny-1)*np.std(y, ddof=1)**2) / dof)

np.random.seed(123)
mu1, mu2, sigma = 170, 170.1, 5

# 2. 작은 표본(n=30)으로 검정
sample1_small = np.random.normal(mu1, sigma, 30)
sample2_small = np.random.normal(mu2, sigma, 30)

# scipy.stats.ttest_ind를 사용하여 두 작은 표본 간의 t-검정을 수행하세요.
t_stat_small, p_value_small = stats.ttest_ind(sample1_small, sample2_small, equal_var=True)
effect_size_small = cohen_d(sample1_small, sample2_small)

print(f"--- 작은 표본 (n=30) 결과 ---")
print(f"P-value: {p_value_small:.4f}")
print(f"Effect Size (Cohen's d): {effect_size_small:.4f}\n")

# 3. 클 표본(n=10000)으로 검정
sample1_large = np.random.normal(mu1, sigma, 10000)
sample2_large = np.random.normal(mu2, sigma, 10000)

# scipy.stats.ttest_ind를 사용하여 두 큰 표본 간의 t-검정을 수행하세요.
t_stat_large, p_value_large = stats.ttest_ind(sample1_large, sample2_large, equal_var=True)
effect_size_large = cohen_d(sample1_large, sample2_large)

print(f"--- 작은 표본 (n=10000) 결과 ---")
print(f"P-value: {p_value_large:.4f}")
print(f"Effect Size (Cohen's d): {effect_size_large:.4f}\n")

# 4. 결과 해석 (아래 주석에 직접 작성해 보세요)
# 표본 크기가 커지자 p-value는 어떻게 변했나요?:
# 작은 표본(n=30)에서는 p-value = 0.7100, 큰 표본(n=10000)에서는 p-value = 0.1673으로 둘 다 유의수준 0.05보다 크다.
# 즉, 이번 시뮬레이션에서는 표본 크기가 커져도 유의하게 나오진 않았지만, 큰 표본에서 p-value가 더 작아진 경향은 보인다.

# 두 경우의 효과 크기는 비슷한가요, 다른가요?:
# Cohen's d는 작은 표본에서 -0.0965, 큰 표본에서 -0.0195로 둘 다 0에 가까운 매우 작은 효과 크기다. 
# 즉, 표본 크기와 관계없이 두 집단 간 차이는 실질적으로 거의 없다.

# 이 결과는 우리에게 무엇을 알려주나요?
# p-value는 표본 크기에 따라 변동성이 크다. 표본이 커질수록 아주 작은 차이도 통계적으로 유의해질 수 있고, 표본이 작을 때는 실제 차이가 있어도 놓칠 수 있다.
# 따라서, 데이터 분석에서는 p-value만 보는 것이 아니라, 효과 크기(실질적 차이의 크기)를 함께 확인해야 한다는 점을 보여준다.

--- 작은 표본 (n=30) 결과 ---
P-value: 0.7100
Effect Size (Cohen's d): -0.0965

--- 작은 표본 (n=10000) 결과 ---
P-value: 0.1673
Effect Size (Cohen's d): -0.0195



### 🤔 생각해 볼 문제:

A/B 테스트에서 웹사이트 버튼 색깔을 바꿨더니, 클릭률이 0.1%p 증가했고, p-value는 0.001이었습니다. 이 결과를 보고 버튼 색깔을 바꾸는 것이 항상 옳다고 할 수 있을까요? '통계적 유의성'과 '실무적 중요성(practical significance)'의 차이에 대해 효과 크기 개념을 바탕으로 설명해 보세요.

- **통계적 유의성 (statistical significance)**  
  p-value가 작다는 건, 관찰된 차이가 단순한 우연일 가능성이 낮다는 뜻이다.  
  즉, 클릭률 차이가 실제로 존재할 가능성이 크다는 것까지는 말해준다.

- **실무적 중요성 (practical significance)**  
  하지만 클릭률 증가폭이 단 0.1%p라면, 실제 비즈니스 측면에서 그 차이가 얼마나 의미 있는지는 별개의 문제다.  
  예를 들어, 수익 구조, 전환율 규모, 변경 비용(디자인, 개발, 테스트 배포 등)을 고려했을 때 0.1%p가 무시해도 될 정도의 미세한 차이일 수 있다.  

- **효과 크기 (effect size)**  
  효과 크기는 단순히 "차이가 있냐/없냐"가 아니라 "그 차이가 얼마나 크고 의미 있냐"를 보여준다.  
  이번 사례처럼 표본 수가 크면 아주 미세한 차이도 p-value를 작게 만들 수 있다. 따라서 p-value만 볼 게 아니라 효과 크기를 계산하고, 그것이 실질적으로 유의미한 수준인지 확인해야 한다.  

👉 정리하면, **통계적으로 유의한 차이가 실무적으로도 중요한 차이를 의미하는 건 아니다.**  
A/B 테스트 의사결정에서는 **p-value + 효과 크기 + 비용·효익 맥락**을 함께 고려해야 한다.
